This file tries to work out how to use PySMT to do some useful stuff

In [1]:
#from pysmt.shortcuts import (
#    Symbol, BOOL, INT, REAL,
#    Bool, Int, Real,
#    TRUE, FALSE, And, Or, Not,
#    GT, GE, LE, LT, Equals, NotEquals,
#    Plus, Minus, Times, Div   
#)
import pysmt
from pysmt import shortcuts as SMT

In [2]:
def _get_smt_solver(logic = None):
    factory = pysmt.factory.Factory(pysmt.shortcuts.get_env())
    slvs    = factory.all_solvers()
    if len(slvs) == 0: raise OSError("Could not find any SMT solvers")
    return pysmt.shortcuts.Solver(name=next(iter(slvs)),logic=logic)
SLV = _get_smt_solver()

We'll want to use the following major functions to manage the state of the solver:

* `SLV.reset_assertions()` - to reset from other possible invocations
* `SLV.add_assertion(formula)` - to add preconditions
* `SLV.push()` and `SLV.pop()` - to manage contexts of preconditions
* `SLV.is_valid(formula)` - to check a goal
* `SLV.solve()` - to check whether the current set of assumptions is satisfiable

In [3]:
SLV.reset_assertions()

A simple example from the PySMT documentation

In [4]:
varA    = SMT.Symbol("A") # default var type is Boolean
varB    = SMT.Symbol("B")
f       = SMT.And(varA, SMT.Not(varB))
f

(A & (! B))

In [5]:
SLV.is_sat(f)

True

In [6]:
g = f.substitute({varB: varA})
g

(A & (! A))

In [7]:
SLV.is_sat(g)

False

Let's show that linear integer problems work correctly with a very simple example.

In [8]:
SLV.reset_assertions()

In [9]:
i, j    = SMT.Symbol("i",SMT.INT), SMT.Symbol("j",SMT.INT)
SLV.push()
SLV.add_assertion( SMT.Equals(i, SMT.Times(SMT.Int(2),j)) )
SLV.add_assertion( SMT.And(SMT.GE(i, SMT.Int(0)), SMT.LT(i, SMT.Int(10))) )
for j_hi in range(0,10): print(j_hi, SLV.is_valid( SMT.LT(j, SMT.Int(j_hi)) ))
print("-----")
for i_const in range(-2,12): print(i_const, SLV.is_sat( SMT.Equals(i, SMT.Int(i_const)) ))
SLV.pop()

0 False
1 False
2 False
3 False
4 False
5 True
6 True
7 True
8 True
9 True
-----
-2 False
-1 False
0 True
1 False
2 True
3 False
4 True
5 False
6 True
7 False
8 True
9 False
10 False
11 False


Yup, looks like they work.  Keep in mind that we need to use this particular `SMT.INT` type, and `SMT.Int` lifts constants into formulae constants of the appropriate type.

In [11]:
x = SMT.Symbol("x$1",SMT.INT)
x

x$1

In [35]:
SMT.INT == INT

True